# Script for a period

In [196]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time


load_dotenv()

True

In [199]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [200]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [63]:
data_giper

{'adverts': [{'type': 9,
   'status': 11,
   'count': 5,
   'advert_list': [{'advertId': 22844853,
     'changeTime': '2025-02-27T14:07:48.633319+03:00'},
    {'advertId': 22844893, 'changeTime': '2025-01-31T13:31:32.654625+03:00'},
    {'advertId': 22950974, 'changeTime': '2025-03-03T12:35:20.701855+03:00'},
    {'advertId': 22951078, 'changeTime': '2025-03-03T12:47:32.279379+03:00'},
    {'advertId': 22951454, 'changeTime': '2025-03-03T15:33:24.073868+03:00'}]},
  {'type': 8,
   'status': 7,
   'count': 7,
   'advert_list': [{'advertId': 17741906,
     'changeTime': '2024-06-08T10:38:13.264875+03:00'},
    {'advertId': 17748698, 'changeTime': '2024-06-08T10:42:37.987864+03:00'},
    {'advertId': 17749089, 'changeTime': '2024-06-08T10:41:42.952579+03:00'},
    {'advertId': 17749228, 'changeTime': '2024-06-24T11:12:43.162712+03:00'},
    {'advertId': 17749561, 'changeTime': '2024-06-08T10:38:39.558529+03:00'},
    {'advertId': 23227483, 'changeTime': '2025-02-19T13:01:41.621239+03:00'}

In [152]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
#df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
#df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
#df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
#df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [ ]:
#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

417


### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [153]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Print the list of all chunks
print("Campaign Chunks for WB-GutenTech:", campaign_chunks_guten)
print("Campaign Chunks for WB-ГиперМаркет:", campaign_chunks_giper)
print("Campaign Chunks for WB-KitchenAid:", campaign_chunks_kitchen)
print("Campaign Chunks for WB-Smart-Market:", campaign_chunks_smart)

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, project_name):
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return all_campaign_data

# Fetch campaign data for each project
all_campaign_data_guten = fetch_campaign_data(campaign_chunks_guten, headers_guten, 'WB-GutenTech')
all_campaign_data_giper = fetch_campaign_data(campaign_chunks_giper, headers_giper, 'WB-ГиперМаркет')
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks_kitchen, headers_kitchen, 'WB-KitchenAid')
all_campaign_data_smart = fetch_campaign_data(campaign_chunks_smart, headers_smart, 'WB-Smart-Market')

# Create DataFrames from the campaign data
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

# Sort each DataFrame by 'createTime' in descending order
campaigns_guten = campaigns_guten.sort_values(by='createTime', ascending=False)
campaigns_giper = campaigns_giper.sort_values(by='createTime', ascending=False)
campaigns_kitchen = campaigns_kitchen.sort_values(by='createTime', ascending=False)
campaigns_smart = campaigns_smart.sort_values(by='createTime', ascending=False)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Campaign Chunks for WB-GutenTech: [[7202795, 2851788, 5146621, 7783632, 7202878, 7784050, 7783759, 7783591, 7783708, 21203170, 21203007, 21203075, 21204098, 21204224, 21202745, 19276746, 19276694, 19276556, 21204285, 21204193, 21212715, 21965547, 7202847, 21351587, 20955667, 20039743, 20955474, 16144421, 20955944, 21351310, 20955301, 11296551, 21820825, 12465028, 22123202, 21350648, 20956444, 21386314, 21385572, 622376, 622381, 958839, 788498, 2594357, 622366, 463196, 8280820, 1801052, 953785, 9905391], [964993, 1801455, 480275, 3467141, 953780, 788619, 1801713, 2852177, 1320718, 8331245, 610869, 480548, 610835, 2594580, 1005723, 610787, 622292, 480455, 8079038, 965014, 622420, 2595030, 4558112, 1802589, 622386, 7202673, 622222, 2594976, 3476612, 622441, 788659, 480513, 622432, 463138, 6923850, 622402, 622454, 1801426, 9714386, 610764, 965005, 622297, 8838251, 622371, 480291, 463364, 610778, 622397, 1801574, 1801541], [610857, 8618683, 789205, 1320709, 10012032, 622448, 789210, 463211,

In [154]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-02-19,2025-02-19,Mr Proper_19.02.2025,23408417,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-02-12,2025-03-10,Venus_12.02.2025,23231292,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-02-12,2025-03-10,Always_12.02.2025,23230882,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры на запястье_27.01.2025,22846531,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-01-27,2025-03-12,AND_Тонометры авто_27.01.2025,22846516,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
470,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
471,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
472,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
473,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [156]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2025-03-01",  # Replace with your start date
    "end": "2025-03-10"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [157]:
# Assume campaigns_guten DataFrame is already defined and contains 'advertId'
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_data(campaign_chunks, headers_guten, date_interval)
# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrieved successfully for chunk 1
Data retrieved successfully for chunk 2
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 4: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 5: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Combined Campaign Data


### Giper

In [158]:
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data
# Fetch campaign data
all_campaign_data_giper = fetch_campaign_data(campaign_chunks, headers_giper, date_interval)

Data retrieved successfully for chunk 1


### Kitchen

In [159]:
chunk_size = 100
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks, headers_kitchen, date_interval)


Data retrieved successfully for chunk 1


### Smart

In [160]:
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_smart = fetch_campaign_data(campaign_chunks, headers_smart, date_interval)

Data retrieved successfully for chunk 1


In [161]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,advertId,interval.begin,interval.end,boosterStats
0,5334,240,4.50,5.11,1226.82,13,0,0.00,0,0,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",21212715,2025-03-01,2025-03-10,NaN
1,2,0,0.00,0.00,0.40,0,0,0.00,0,0,"[{'date': '2025-03-07T03:00:00+03:00', 'views'...",21578389,2025-03-01,2025-03-10,NaN
2,69487,1678,2.41,5.40,9063.39,146,35,2.09,35,62542,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",21498159,2025-03-01,2025-03-10,"[{'date': '2025-03-01T00:00:00Z', 'nm': 137609..."
3,13124,947,7.22,3.01,2850.10,72,9,0.95,9,60505,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",19276746,2025-03-01,2025-03-10,NaN
4,28773,1005,3.49,5.77,5802.70,81,22,2.19,22,242310,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",21203007,2025-03-01,2025-03-10,NaN
5,37804,900,2.38,9.52,8564.76,79,15,1.67,15,178270,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",21204224,2025-03-01,2025-03-10,NaN
6,4290,71,1.66,10.14,719.85,7,2,2.82,2,2212,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",23408417,2025-03-01,2025-03-10,"[{'date': '2025-03-01T00:00:00Z', 'nm': 327350..."
7,9077,148,1.63,12.47,1845.64,4,0,0.00,0,0,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",21577713,2025-03-01,2025-03-10,"[{'date': '2025-03-01T00:00:00Z', 'nm': 320765..."
8,82671,2453,2.97,4.83,11836.21,293,55,2.24,55,79035,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",23231292,2025-03-01,2025-03-10,"[{'date': '2025-03-01T00:00:00Z', 'nm': 145759..."
9,37976,863,2.27,10.00,8632.65,64,10,1.16,10,83743,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",21204193,2025-03-01,2025-03-10,NaN


In [162]:
print(campaign_guten_df["advertId"].nunique())

#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

57


In [163]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-03-01 03:00:00,187556878,Триммер для бороды и усов MGK3410,6,0,0.00,0.00,1.38,0,0,0.00,0,0,21212715
1,2025-03-01 03:00:00,245600931,Триммер для бороды и усов BT3421,4,0,0.00,0.00,0.92,0,0,0.00,0,0,21212715
2,2025-03-01 03:00:00,203510173,Триммер для бороды и усов BT5420,4,0,0.00,0.00,0.92,0,0,0.00,0,0,21212715
3,2025-03-01 03:00:00,203505281,Триммер для бороды и усов MGK7420 + бритва Gil...,17,0,0.00,0.00,3.91,0,0,0.00,0,0,21212715
4,2025-03-01 03:00:00,138937204,Триммер для бороды и усов OneTool XT 3100,7,0,0.00,0.00,1.61,0,0,0.00,0,0,21212715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7156,2025-03-10 03:00:00,52469734,Вакууматор для продуктов с пакетами Vacu OneTouch,58,4,6.90,2.45,9.81,0,0,0.00,0,0,18870595
7157,2025-03-10 03:00:00,52418766,Вакууматор для продуктов с запайкой Gourmet VA...,68,2,2.94,5.68,11.35,0,0,0.00,0,0,18870595
7158,2025-03-10 03:00:00,23806943,Вакууматор для продуктов с запайкой VC 11,85,5,5.88,2.68,13.42,0,0,0.00,0,0,18870595
7159,2025-03-10 03:00:00,52441643,Вакууматор для продуктов с запайкой VR 190,56,1,1.79,9.26,9.26,0,0,0.00,0,0,18870595


In [164]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,411,18,2.815000,3.305000,78.32,2,0,0.000000,0,0,21202745,WB-GutenTech,Wildberries
1,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,444,20,4.395000,3.873333,79.94,1,0,0.000000,0,0,21202745,WB-GutenTech,Wildberries
2,25628178,2025-03-01 03:00:00,Бритва мужская электрическая S3 300s,474,17,2.786667,3.190000,83.18,0,0,0.000000,0,0,21202745,WB-GutenTech,Wildberries
3,25725047,2025-03-01 03:00:00,Триммер для бороды и усов BT3222,2,0,0.000000,0.000000,0.10,0,0,0.000000,0,0,21212396,WB-GutenTech,Wildberries
4,25962484,2025-03-01 03:00:00,Кухонные весы Kitchen Energy,1104,17,1.160000,16.356667,170.52,0,0,0.000000,0,0,18870701,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4228,288465481,2025-03-10 03:00:00,Набор Gillette Venus с чехлом для бритвы,1427,93,5.273333,4.076667,208.60,2,1,0.926667,1,1115,23236382,WB-Smart-Market,Wildberries
4229,288465487,2025-03-10 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.000000,0.000000,0.06,0,0,0.000000,0,0,23236144,WB-Smart-Market,Wildberries
4230,313543155,2025-03-10 03:00:00,,1272,18,1.270000,15.510000,164.92,3,2,5.556667,2,6906,23236221,WB-Smart-Market,Wildberries
4231,313543156,2025-03-10 03:00:00,,1542,22,1.120000,17.813333,203.51,4,1,8.333333,1,3664,23236221,WB-Smart-Market,Wildberries


In [165]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,411,18,78.32,2,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
1,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,444,20,79.94,1,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
2,25628178,2025-03-01 03:00:00,Бритва мужская электрическая S3 300s,474,17,83.18,0,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
3,25725047,2025-03-01 03:00:00,Триммер для бороды и усов BT3222,2,0,0.10,0,0,0,0,21212396,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Триммеры_07.11.2024,Идут показы,Автоматическая кампания
4,25962484,2025-03-01 03:00:00,Кухонные весы Kitchen Energy,1104,17,170.52,0,0,0,0,18870701,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Весы кух_2 SKU_25.07.24,Кампания на паузе,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4228,288465481,2025-03-10 03:00:00,Набор Gillette Venus с чехлом для бритвы,1427,93,208.60,2,1,1,1115,23236382,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-03-05,VENUS_Наборы_12.02.2025,Кампания на паузе,Автоматическая кампания
4229,288465487,2025-03-10 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.06,0,0,0,0,23236144,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания
4230,313543155,2025-03-10 03:00:00,,1272,18,164.92,3,2,2,6906,23236221,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания
4231,313543156,2025-03-10 03:00:00,,1542,22,203.51,4,1,1,3664,23236221,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания


In [ ]:
unique = df_final['nmId'].nunique()
unique


91

In [181]:
# Filter rows where Project is 'Wb-GutenTech'
filtered_df = df_final[df_final['Project'] == 'WB-GutenTech']

# Extract unique nmId values from the filtered DataFrame
unique_nmId_values = filtered_df['nmId'].unique().tolist()
print(len(unique_nmId_values))

245


In [203]:
# API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history"

# Assuming filtered_df is already defined and contains the necessary data
# Extract all unique nmId values from the filtered DataFrame
filtered_df = df_final[df_final['Project'] == 'WB-GutenTech']
unique_nmId_values = filtered_df['nmId'].unique().tolist()
print("Total unique nmId values:", len(unique_nmId_values))  # Debugging line

# Function to send a single POST request
def send_request(nm_ids, period, headers):
    request_body = {
        "nmIDs": nm_ids,
        "period": period
    }
    response = requests.post(url, headers=headers, json=request_body)
    return response

# Function to fetch data in batches
def fetch_data_in_batches(nm_ids, period, headers):
    all_data = []
    batch_size = 20  # Maximum allowed nmIDs per request
    requests_per_minute = 3  # Maximum allowed requests per minute
    interval = 60 / requests_per_minute  # Time interval between requests in seconds

    for i in range(0, len(nm_ids), batch_size):
        batch = nm_ids[i:i + batch_size]
        response = send_request(batch, period, headers)

        if response.status_code == 200:
            try:
                data = response.json()
                if not data.get('error') and 'data' in data:
                    all_data.extend(data['data'])
                    print(f"Data retrieved successfully for batch {i // batch_size + 1}")
                else:
                    print(f"Error in response for batch {i // batch_size + 1}: {data.get('errorText', 'No error text')}")
            except ValueError as e:
                print(f"Failed to decode JSON for batch {i // batch_size + 1}: {e}")
        else:
            print(f"Error for batch {i // batch_size + 1}: {response.status_code}, {response.text}")

        time.sleep(interval)  # Respect the rate limit

    return all_data

# Define the period for the report
period = {
    "begin": "2025-03-06",
    "end": "2025-03-11"
}

# Fetch data for all unique nmId values
all_data = fetch_data_in_batches(unique_nmId_values, period, headers_guten)

# Process the data as needed
# Flatten the nested 'history' data for easier analysis
flattened_data = []
for item in all_data:
    nmID = item['nmID']
    imtName = item['imtName']
    vendorCode = item['vendorCode']
    for history in item['history']:
        history_entry = {
            'nmID': nmID,
            'imtName': imtName,
            'vendorCode': vendorCode,
            'dt': history['dt'],
            'openCardCount': history['openCardCount'],
            'addToCartCount': history['addToCartCount'],
            'addToCartConversion': history['addToCartConversion'],
            'ordersCount': history['ordersCount'],
            'ordersSumRub': history['ordersSumRub'],
            'cartToOrderConversion': history['cartToOrderConversion'],
            'buyoutsCount': history['buyoutsCount'],
            'buyoutsSumRub': history['buyoutsSumRub'],
            'buyoutPercent': history['buyoutPercent']
        }
        flattened_data.append(history_entry)

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data)
print(df)



Total unique nmId values: 245
Data retrieved successfully for batch 1
Data retrieved successfully for batch 2
Data retrieved successfully for batch 3
Data retrieved successfully for batch 4
Data retrieved successfully for batch 5
Data retrieved successfully for batch 6
Data retrieved successfully for batch 7
Data retrieved successfully for batch 8
Data retrieved successfully for batch 9
Data retrieved successfully for batch 10
Data retrieved successfully for batch 11
Data retrieved successfully for batch 12
Data retrieved successfully for batch 13
           nmID                                            imtName vendorCode  \
0      25725047                   Триммер для бороды и усов BT3222   Б0046733   
1      25725047                   Триммер для бороды и усов BT3222   Б0046733   
2      25725047                   Триммер для бороды и усов BT3222   Б0046733   
3      25725047                   Триммер для бороды и усов BT3222   Б0046733   
4      25725047                   Триммер

In [202]:
df

,nmID,imtName,vendorCode,dt,openCardCount,addToCartCount,addToCartConversion,ordersCount,ordersSumRub,cartToOrderConversion,buyoutsCount,buyoutsSumRub,buyoutPercent
0,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-06,48,2,4,0,0,0,0,0,0
1,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-07,45,2,4,0,0,0,0,0,0
2,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-08,20,0,0,0,0,0,0,0,0
3,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-09,48,1,2,1,7340,100,0,0,0
4,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-10,47,3,6,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,181893223,Набор ProGlide Power бритва с 1 кассетой и гел...,8700216171700,2025-03-07,15,2,13,1,3185,50,1,3185,100
596,181893223,Набор ProGlide Power бритва с 1 кассетой и гел...,8700216171700,2025-03-08,13,0,0,0,0,0,0,0,0
597,181893223,Набор ProGlide Power бритва с 1 кассетой и гел...,8700216171700,2025-03-09,15,0,0,0,0,0,0,0,0
598,181893223,Набор ProGlide Power бритва с 1 кассетой и гел...,8700216171700,2025-03-10,16,1,6,0,0,0,0,0,0


In [204]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final['day'] = pd.to_datetime(df_final['day']).dt.date
df['dt'] = pd.to_datetime(df['dt']).dt.date

# Rename columns in df2 to match df1 for merging
df.rename(columns={'nmID': 'nmId', 'dt': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df = pd.merge(
    df_final,
    df[['nmId', 'day', 'ordersCount', 'ordersSumRub']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df['ordersCount'].fillna(0, inplace=True)
merged_df['ordersSumRub'].fillna(0, inplace=True)

# Display the merged DataFrame
merged_df

C:\Users\User\AppData\Local\Temp\ipykernel_3956\2202212359.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_3956\2202212359.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub
0,25623718,2025-03-01,Бритва мужская электрическая S3 300BT с триммером,411,18,78.32,2,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,0.0,0.0
1,25624340,2025-03-01,Электробритва Series 3 300s,444,20,79.94,1,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,0.0,0.0
2,25628178,2025-03-01,Бритва мужская электрическая S3 300s,474,17,83.18,0,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,0.0,0.0
3,25725047,2025-03-01,Триммер для бороды и усов BT3222,2,0,0.10,0,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Триммеры_07.11.2024,Идут показы,Автоматическая кампания,0.0,0.0
4,25962484,2025-03-01,Кухонные весы Kitchen Energy,1104,17,170.52,0,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Весы кух_2 SKU_25.07.24,Кампания на паузе,Автоматическая кампания,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4228,288465481,2025-03-10,Набор Gillette Venus с чехлом для бритвы,1427,93,208.60,2,1,1,1115,...,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-03-05,VENUS_Наборы_12.02.2025,Кампания на паузе,Автоматическая кампания,0.0,0.0
4229,288465487,2025-03-10,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.06,0,0,0,0,...,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания,0.0,0.0
4230,313543155,2025-03-10,,1272,18,164.92,3,2,2,6906,...,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания,0.0,0.0
4231,313543156,2025-03-10,,1542,22,203.51,4,1,1,3664,...,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания,0.0,0.0


## Insert the data

In [205]:
password = os.getenv('ClickHouse')

In [206]:
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                      # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [212]:
# Define the table name
table_name = 'campaign_data_wb'

# Ensure date columns are in datetime format
df_final['day'] = pd.to_datetime(df_final['day'], format='%Y-%m-%d %H:%M:%S')
df_final['endTime'] = pd.to_datetime(df_final['endTime']).dt.date
df_final['createTime'] = pd.to_datetime(df_final['createTime']).dt.date
df_final['startTime'] = pd.to_datetime(df_final['startTime']).dt.date

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_final.to_numpy()]

# Define the column names
column_names = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'ordersCount', 'ordersSumRub', 'sum_price', 'advertId', 'Project', 'Marketplace',
    'endTime', 'createTime', 'startTime', 'name_campaign', 'status', 'type'
]

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=column_names)

print("Data inserted successfully!")

TypeError: object of type 'datetime.date' has no len()